In [14]:
using Devectorize

INFO: Precompiling module Devectorize...


In [16]:
function adjust!(a::Float64,x::Array{Float64,1},y::Array{Float64,1})
    n=length(y)
    for i=1:n
        y[i]=y[i]+a*x[i]
    end
end

adjust! (generic function with 1 method)

In [19]:
function foo1(n,nIter)
    y = zeros(n)
    for i=1:nIter
        for j=1:n
            y[j] = y[j] + 1.0
        end
    end
    return y
end

foo1 (generic function with 1 method)

In [17]:
function foo2(n,nIter)
    y = zeros(n)
    J = ones(n)
    for i=1:nIter
        y += J
    end
    return y
end

foo2 (generic function with 1 method)

In [18]:
function foo2a(n,nIter)
    y = zeros(n)
    J = ones(n)
    for i=1:nIter
        @devec y += J
    end
    return y
end

foo2a (generic function with 1 method)

In [20]:
function foo3(n,nIter)
    y = zeros(n)
    J = ones(n)
    for i=1:nIter
        BLAS.axpy!(1.0,J,y)
    end
    return y
end

foo3 (generic function with 1 method)

In [29]:
function foo4(n,nIter)
    y = zeros(n)
    J = ones(n)
    for i=1:nIter
        adjust!(1.0,J,y)
    end
    return y
end

foo4 (generic function with 1 method)

In [24]:
@time foo1(1000000,2000);

  2.526550 seconds (6 allocations: 7.630 MB, 0.09% gc time)


In [23]:
@time foo2(1000000,2000);

  7.700268 seconds (8.01 k allocations: 14.916 GB, 27.96% gc time)


In [26]:
@time foo2a(1000000,2000);

  7.457856 seconds (8.01 k allocations: 14.916 GB, 20.41% gc time)


In [26]:
@time foo3(1000000,2000);

  1.651654 seconds (8 allocations: 15.259 MB, 0.12% gc time)


In [31]:
@time foo4(1000000,2000);

  2.892965 seconds (8 allocations: 15.259 MB, 0.04% gc time)


In [1]:
nrow=1000000
ncol=1000
X=randn(nrow,ncol);

In [2]:
sol=randn(ncol)
ycorr=zeros(nrow);

In [35]:
@time BLAS.axpy!(-1.0,X*sol,ycorr);

  0.004169 seconds (8 allocations: 78.375 KB)


In [3]:
function test(X,sol,ycorr,niter)
    for i =1:niter
        BLAS.axpy!(-1.0,X*sol,ycorr)
    end
end

test (generic function with 1 method)

In [4]:
function test2(X,sol,ycorr,niter)
    for i =1:niter
        ycorr-=X*sol
    end
end

test2 (generic function with 1 method)

In [5]:
function test3(X,sol,ycorr,niter)
    for i =1:niter
        #sol = randn(ncol)
        BLAS.gemv!('N',1.0,X,sol,1.0,ycorr)
    end
end

test3 (generic function with 1 method)

In [8]:
niter=100
@time test(X,sol,ycorr,niter)

 57

In [10]:
niter=100
@time test2(X,sol,ycorr,niter)

 57.982593 seconds (8.09 k allocations: 1.490 GB, 0.24% gc time)


In [9]:
niter=100
@time test3(X,sol,ycorr,niter)

.274542 seconds (404 allocations: 762.949 MB, 0.13% gc time)
 56.873426 seconds (4 allocations: 160 bytes)


In [ ]:
gemv!(tA, alpha, A, x, beta, y)

In [59]:
BLAS.gemv!('N',1.0,X,sol,1.0,ycorr)

10000-element Array{Float64,1}:
  10.7252  
  12.206   
   6.73659 
  32.4708  
  18.3474  
   7.82986 
 -17.7365  
  27.5821  
 -17.2379  
 -41.027   
  56.5367  
 -54.0944  
  14.4407  
   ⋮       
  14.239   
  45.5786  
 -19.22    
  -9.08581 
   9.34264 
  43.6002  
  39.4071  
 -24.8009  
  -0.992228
  28.9838  
  -6.73441 
   4.22691 